# Imports

In [1]:
import numpy as np

In [2]:
import os.path as osp
import numpy as np
from tqdm import tqdm
from utils import pickle_save, pickle_load
from pprint import pprint
from utils.data.delf import datum_io
from copy import deepcopy

In [3]:
import sacred
from sacred import SETTINGS
from sacred.utils import apply_backspaces_and_linefeeds
from numpy import linalg as LA
from utils import pickle_load, pickle_save
#from utils.revisited import compute_metrics
from utils.data.delf import datum_io

In [4]:
ex = sacred.Experiment('Prepare Top-K (VIQUAE FOR RTT)', interactive=True)
# Filter backspaces and linefeeds
SETTINGS.CAPTURE_MODE = 'sys'
ex.captured_out_filter = apply_backspaces_and_linefeeds

In [5]:
feature_name = 'r50_gldv1'
set_name = 'test'
gnd_name = 'gnd_'+ set_name+'.pkl'

In [6]:
dataset_name = 'viquae_for_rrt'
data_dir = osp.join('/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data', dataset_name)

In [7]:
use_aqe = False
aqe_params = {'k': 2, 'alpha': 0.3}

save_nn_inds = True

In [8]:
with open(osp.join(data_dir,  set_name+'_query.txt')) as fid:
    query_lines   = fid.read().splitlines()

In [9]:
len(query_lines)

1257

In [10]:
with open(osp.join(data_dir, set_name+'_gallery.txt')) as fid:
    gallery_lines = fid.read().splitlines()

In [11]:
query_feats = []
for i in tqdm(range(len(query_lines))):
    name = osp.splitext(osp.basename(query_lines[i].split(';;')[0]))[0]
    path = osp.join(data_dir, 'delg_' + feature_name, name + '.delg_global')
    query_feats.append(datum_io.ReadFromFile(path))

100%|██████████| 1257/1257 [00:07<00:00, 168.01it/s]


In [12]:
query_feats = np.stack(query_feats, axis=0)
query_feats = query_feats / LA.norm(query_feats, axis=-1)[:, None]

In [13]:
query_feats.shape

(1257, 2048)

In [14]:
selection_lines = np.genfromtxt('/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt/'+
                                set_name+'_selection_imgs.txt', dtype='str')
selection_lines.shape

(1257, 100)

In [15]:
selection_lines[0][:10]

array(['512px-Dannii_Minogue_arrives_at_the_58th_Annual_Logie_Awards_at_Crown_Palladium_(26904220225)_cropped.jpg',
       "512px-Back_o'_th'_Bank_House,_Hereford_St,_Bolton_-_geograph.org.uk_-_158440.jpg",
       '512px-Harry_Hill_(crop).jpg', '512px-Macys_dep_store.JPG',
       '512px-John_Lewis_&_Partners_Oxford_Street_store.jpg',
       '512px-John_Lewis_&_Partners_Oxford_Street_store.jpg',
       '512px-Andy_Williams_1963.JPG',
       '512px-Taken_by_Trees_Popaganda_2013_(cropped).jpg',
       '512px-Poison_logo.png',
       '512px-Warner_Records_(2019_Logo).svg.png'], dtype='<U233')

In [16]:
# wiki_img   = '.'.join((wiki_item['image'].split('.')[:-1]))

In [17]:
selection_index_feats = []
for i in tqdm(range(len(selection_lines))):
    index_feats = []
    for image_file in selection_lines[i]:
        name = '.'.join((image_file.split('.')[:-1]))
        path = osp.join(data_dir, 'delg_' + feature_name, name + '.delg_global')
        index_feats.append(datum_io.ReadFromFile(path))
    selection_index_feats.append(index_feats)

100%|██████████| 1257/1257 [07:47<00:00,  2.69it/s]


In [18]:
min([len(selection_index_feats[i]) for i in range(len(selection_index_feats))])

100

In [19]:
selection_index_feats = np.stack(selection_index_feats, axis=0)
selection_index_feats.shape

(1257, 100, 2048)

In [20]:
query_feats[0].shape

(2048,)

In [21]:
selection_sims = []
for i in range(len(selection_index_feats)):
    index_feats = np.stack(selection_index_feats[i], axis=0)
    index_feats = index_feats / LA.norm(index_feats, axis=-1)[:, None]
    selection_sims.append(np.matmul(query_feats[i], index_feats.T))

In [22]:
sims = np.array(selection_sims)
sims.shape

(1257, 100)

In [23]:
sims

array([[ 0.19043577, -0.06568643,  0.4883389 , ...,  0.22986387,
         0.22986382,  0.30038485],
       [ 0.43800008,  0.43800008,  0.4462106 , ...,  0.23238036,
         0.20802978,  0.20802978],
       [ 0.7125816 ,  0.7125816 ,  0.7125816 , ...,  0.30701733,
         0.7248775 ,  0.7248775 ],
       ...,
       [ 0.6444179 ,  0.31371546,  0.28090614, ...,  0.17109555,
         0.35594177,  0.46608672],
       [-0.00440816, -0.11441729,  0.08888464, ...,  0.02028872,
        -0.03921147,  0.04146704],
       [-0.02579766, -0.02579766, -0.05192305, ...,  0.06185038,
        -0.0631685 , -0.02348739]], dtype=float32)

In [24]:
if use_aqe:
    alpha = aqe_params['alpha']
    nn_inds = np.argsort(-sims, -1)
    query_aug = deepcopy(query_feats)
    for i in range(len(query_feats)):
        new_q = [query_feats[i]]
        for j in range(aqe_params['k']):
            nn_id = nn_inds[i, j]
            weight = sims[i, nn_id] ** aqe_params['alpha']
            new_q.append(weight * index_feats[nn_id])
        new_q = np.stack(new_q, 0)
        new_q = np.mean(new_q, axis=0)
        query_aug[i] = new_q/LA.norm(new_q, axis=-1)
    sims = np.matmul(query_aug, index_feats.T)

In [25]:
selection_index_feats[0].shape

(100, 2048)

In [26]:
nn_inds = np.argsort(-sims, -1)
nn_dists = deepcopy(sims)
for i in range(query_feats.shape[0]):
    index_feats = selection_index_feats[i]
    for j in range(index_feats.shape[0]):
        nn_dists[i, j] = sims[i, nn_inds[i, j]]

In [27]:
nn_inds.shape

(1257, 100)

In [28]:
if save_nn_inds:
    output_path = osp.join(data_dir, set_name + '_nn_inds_%s.pkl' % feature_name)
    pickle_save(output_path, nn_inds)

In [29]:
def compute_ap(ranks, nres):
    """
    Computes average precision for given ranked indexes.
    
    It assumes that `ranks` contains the ranks for all expected positive
    index images to be retrieved. If `positive_ranks` is empty, returns
    `average_precision` = 0.
    Note that average precision computation here does NOT use the finite sum
    method (see
    https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Average_precision)
    which is common in information retrieval literature. Instead, the method
    implemented here integrates over the precision-recall curve by averaging two
    adjacent precision points, then multiplying by the recall step. This is the
    convention for the Revisited Oxford/Paris datasets.
    
    Arguments
    ---------
    ranks : zerro-based ranks of positive images
    nres  : number of positive images
    
    Returns
    -------
    ap    : average precision
    
    """

    # number of images ranked by the system
    nimgranks = len(ranks)

    # accumulate trapezoids in PR-plot
    ap = 0

    recall_step = 1. / nres

    for j in np.arange(nimgranks):
        rank = ranks[j]

        if rank == 0:
            precision_0 = 1.
        else:
            precision_0 = float(j) / rank

        precision_1 = float(j + 1) / (rank + 1)

        ap += (precision_0 + precision_1) * recall_step / 2.

    return ap

In [30]:
def compute_map(ranks, gnd, kappas=[]):
    """
    Computes the mAP for a given set of returned results.

         Usage: 
           map = compute_map (ranks, gnd) 
                 computes mean average precsion (map) only
        
           map, aps, pr, prs = compute_map (ranks, gnd, kappas) 
                 computes mean average precision (map), average precision (aps) for each query
                 computes mean precision at kappas (pr), precision at kappas (prs) for each query
        
         Notes:
         1) ranks starts from 0, ranks.shape = db_size X #queries
         2) The junk results (e.g., the query itself) should be declared in the gnd stuct array
         3) If there are no positive images for some query, that query is excluded from the evaluation
    """

    map = 0.
    nq = len(gnd) # number of queries
    aps = np.zeros(nq)
    pr = np.zeros(len(kappas))
    prs = np.zeros((nq, len(kappas)))
    nempty = 0

    for i in np.arange(nq):
        qgnd = np.array(gnd[i]['ok'])
        qgndj = np.array(gnd[i]['junk'])

        # no positive groundtruth images, skip from the average
        if qgnd.shape[0] == 0:
            aps[i] = float('nan')
            prs[i, :] = float('nan')
            nempty += 1
            continue

        # only negative images retrieved in the IR step
        if qgnd.shape[0] > 100:
            aps[i] = 0.
            prs[i, :] = 0.
            continue
        
        # sorted positions of positive and junk images (0 based)
        pos  = np.arange(ranks.shape[0])[np.isin(ranks[:,i], qgnd)]
        junk = np.arange(ranks.shape[0])[np.isin(ranks[:,i], qgndj)]

        k = 0
        ij = 0
        if len(junk):
            # decrease positions of positives based on the number of
            # junk images appearing before them
            ip = 0
            while (ip < len(pos)):
                while (ij < len(junk) and pos[ip] > junk[ij]):
                    k += 1
                    ij += 1
                pos[ip] = pos[ip] - k
                ip += 1

        # compute ap
        ap = compute_ap(pos, len(qgnd))
        map = map + ap
        aps[i] = ap

        # compute precision @ k
        pos += 1 # get it to 1-based
        for j in np.arange(len(kappas)):
            if len(pos) == 0:
                max_pos = kappas[j]
            else: max_pos = max(pos)
            kq = min(max_pos, kappas[j]); 
            prs[i, j] = (pos <= kq).sum() / kq
        pr = pr + prs[i, :]

    map = map / nq #(nq - nempty)
    pr = pr /nq # (nq - nempty)

    return map, aps, pr, prs, nempty

In [31]:
gnd_data = pickle_load(osp.join(data_dir, gnd_name))

In [32]:
nn_inds.T[:,1], gnd_data['gnd'][1]['hard']

(array([68, 66, 85, 15, 90,  9,  8, 64, 30,  2, 78, 60,  1,  0, 22, 65, 87,
        29, 34, 72, 16, 27, 88, 84, 83, 71, 70, 21, 94, 28, 31, 17, 32, 13,
        14, 47, 81, 82, 93, 44, 67, 42, 43, 58, 59, 57, 38, 37, 36, 39, 50,
        45, 18, 96, 97, 86, 19, 49, 80, 79, 99, 98, 69, 33, 63, 35, 92, 23,
        24, 48, 52, 51, 20, 95, 41, 25, 26, 46, 12, 11, 75, 74, 73, 76,  6,
         7,  3,  5,  4, 40, 55, 53, 56, 54, 91, 62, 77, 61, 10, 89]),
 [96, 36, 34, 88, 0, 40, 64, 98, 60, 87, 97])

In [33]:
np.where(nn_inds.T[:,0]==10)

(array([68]),)

In [34]:
pos  = np.arange(nn_inds.T.shape[0])[np.isin(nn_inds.T[:,0], gnd_data['gnd'][0]['hard'])]
junk = np.arange(nn_inds.T.shape[0])[np.isin(nn_inds.T[:,0], gnd_data['gnd'][0]['junk'])]
pos, junk

(array([ 1, 17, 93]),
 array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
        53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
        70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,
        87, 88, 89, 90, 91, 92, 94, 95, 96, 97, 98, 99]))

In [35]:
def compute_metrics(dataset, ranks, gnd, kappas=[1, 5, 10]):
    print(ranks.shape)
    
    # old evaluation protocol
    if dataset.startswith('classic'):
        map, aps, _, _ = compute_map(ranks, gnd)
        out = {'map': np.around(map*100, decimals=3)}
        print('>> {}: mAP {:.2f}'.format(dataset, out['map']))

    # new evaluation protocol
    elif dataset.startswith('viquae'):
        
        gnd_t = []
        for i in range(len(gnd)):
            g = {}
            g['ok'] = np.concatenate([gnd[i]['hard']])
            g['junk'] = np.concatenate([gnd[i]['junk']])
            gnd_t.append(g)
        mapH, apsH, mprH, prsH, nempty = compute_map(ranks, gnd_t, kappas)


        out = {
            'H_map': np.around(mapH*100, decimals=2),
            'H_mp':  np.around(mprH*100, decimals=2),
        }

        print('>> {}: mAP H: {}'.format(dataset, out['H_map']))
        print('>> {}: mP@k{} H: {}'.format(dataset, kappas, out['H_mp']))

    return out, mapH, apsH, mprH, prsH, nempty

In [36]:
np.array(gnd_data['gnd'][10]['junk']).shape[0]

69

In [37]:
np.arange(nn_inds.T.shape[0])[np.in1d(nn_inds.T[:,i], gnd_data['gnd'][10]['hard'])], gnd_data['gnd'][10]['hard']

(array([ 1,  2, 11, 12, 14, 16, 30, 32, 33, 35, 39, 40, 46, 48, 49, 55, 59,
        70, 73, 76, 77, 80, 85, 87, 88, 89, 90, 92, 96, 97, 99]),
 [33,
  2,
  54,
  16,
  13,
  27,
  19,
  39,
  43,
  30,
  10,
  40,
  73,
  55,
  71,
  15,
  47,
  81,
  45,
  20,
  83,
  18,
  46,
  17,
  35,
  67,
  1,
  95,
  28,
  49,
  48])

In [38]:
out, mapH, apsH, mprH, prsH, nempty = compute_metrics('viquae', nn_inds.T, gnd_data['gnd'][:5], kappas=[1,5,6,10,100])

(100, 1257)
>> viquae: mAP H: 100.0
>> viquae: mP@k[1, 5, 6, 10, 100] H: [100. 100. 100. 100. 100.]


In [39]:
prsH

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [40]:
from ranx import Qrels, Run

/mnt/beegfs/home/smessoud/anaconda3/envs/rrt/lib/python3.7/site-packages/ranx/qrels_run_common.py:7: UserWarning: Sorting disabled. Assumes that you provided sorted doc_ids!
  warnings.warn("Sorting disabled. Assumes that you provided sorted doc_ids!")


In [41]:
qrels_dict = {}
run_dict = {}
for i in range(nn_inds.shape[0]):
    q_str = 'q_' + str(i)
    
    qrels_dict[q_str] = dict([('d_' + str(i) + '_' + str(key), 1) for key in gnd_data['gnd'][i]['hard']])
    run_dict[q_str] = dict([('d_' + str(i) + '_' + str(key), 1) for key in nn_inds.T[:,i]])

In [42]:
from ranx import Qrels, Run

qrels = Qrels(qrels_dict)
run = Run(run_dict)

In [43]:
from ranx import evaluate

evaluate(qrels, run, ["map", "map@1", "map@5", "map@10",
                      "mrr", "mrr@1", "mrr@5", "mrr@10",
                      "precision", "precision@1", "precision@5", "precision@10"])
# ValueError: Metric  not supported. Supported metrics are `hits`, `hit_rate`,
# `precision`, `recall`, `f1`, `r-precision`, `mrr`, `map`, `ndcg`, and `ndcg_burges`.

{'map': 0.14074134322740486,
 'map@1': 0.010973932258896897,
 'map@5': 0.03231668106916029,
 'map@10': 0.044352008882014717,
 'mrr': 0.18898238538263756,
 'mrr@1': 0.10978520286396182,
 'mrr@5': 0.16284805091487667,
 'mrr@10': 0.17362010834564534,
 'precision': 0.09799522673031026,
 'precision@1': 0.10978520286396182,
 'precision@5': 0.10389817024661895,
 'precision@10': 0.0968178202068417}